### Secondly, note that there are two sources of data: historical and live. Many statistical models assume that the distribution of data stays the same over time. (The technical term is that the distribution is stationary.) But in practice, the distribution of data changes over time, sometimes drastically. This is called distribution drift. As an example, think about building a recommender for news articles. The trending topics change every day, sometimes every hour; what was popular yesterday may no longer be relevant today. One can imagine the distribution of user preference for news articles changing rapidly over time. Hence it’s important to be able to detect distribution drift and adapt the model accordingly.

### One way to detect distribution drift is to continue to track the model’s performance on the validation metric on live data. If the performance is comparable to the validation results when the model was built, then the model still fits the data. When performance starts to degrade, then it’s probable that the distribution of live data has drifted sufficiently from historical data, and it’s time to retrain the model. Monitoring for distribution drift is often done “offline” from the production environment. Hence we are grouping it into offline evaluation.



## Log Loss

### The beautiful thing about this definition is that it is intimately tied to information theory: log-loss is the cross entropy between the distribution of the true labels and the predictions, and it is very closely related to what’s known as the relative entropy, or Kullback–Leibler divergence. Entropy measures the unpredictability of something. Cross entropy incorporates the entropy of the true distribution, plus the extra unpredictability when one assumes a different distribution than the true distribution. So log-loss is an information-theoretic measure to gauge the “extra noise” that comes from using a predictor as opposed to the true labels. By minimizing the cross entropy, we maximize the accuracy of the classifier.

## AUC VS ROC

###  The ROC curve shows the sensitivity of the classifier by plotting the rate of true positives to the rate of false positives (see Figure 2-2). In other words, it shows you how many correct positive classifications can be gained as you allow for more and more false positives. The perfect classifier that makes no mistakes would hit a true positive rate of 100% immediately, without incurring any false positives—this almost never happens in practice.

### The ROC curve is not just a single number; it is a whole curve. It provides nuanced details about the behavior of the classifier, but it’s hard to quickly compare many ROC curves to each other. In particular, if one were to employ some kind of automatic hyperparameter tuning mechanism (a topic we will cover in Chapter 4), the machine would need a quantifiable score instead of a plot that requires visual inspection. The AUC is one way to summarize the ROC curve into a single number, so that it can be compared easily and automatically. A good ROC curve has a lot of space under it (because the true positive rate shoots up to 100% very quickly). A bad ROC curve covers very little area. So high AUC is good, and low AUC is not so good.

## A & B Testing

### This machinery is known as statistical hypothesis testing. It decides between a null hypothesis and an alternate hypothesis. Most of the time, A/B tests are formulated to answer the question, “Does this new model lead to a statistically significant change in the key metric?” The null hypothesis is often “the new model doesn’t change the average value of the key metric,” and the alternative hypothesis “the new model changes the average value of the key metric.” The test for the average value (the population mean, in statistical speak) is the most common, but there are tests for other population parameters as well.

### Briefly, A/B testing involves the following steps:

- Split into randomized control/experimentation groups.
- Observe behavior of both groups on the proposed methods.
- Compute test statistics.
- Compute p-value.
- Output decision.

## Pitfalls of A/B Testing

### Complete Separation of Experiences
- First, take a look at your user randomization and group splitting module. Does it cleanly split off a portion of your users for the experimentation group? Are they experiencing only the new design (or model, or whatever)?
- It’s important to cleanly and completely separate the experiences between the two groups. Suppose you are testing a new button for your website. If the button appears on every page, then make sure the same user sees the same button everywhere. It’ll be better to split by user ID (if available) or user sessions instead of individual page visits.
- Also watch out for the possibility that some of your users have been permanently “trained” by the old model or design and prefer the way things were before. In their KDD 2012 paper, Kohavi et al. calls this the carryover effect. Such users carry the “baggage of the old” and may return biased answers for any new model. If you think this might be the case, think about acquiring a brand new set of users or randomizing the test buckets.

### Which Metric
- The next important question is, on which metric should you evaluate the model? Ultimately, the right metric is probably a business metric. But this may not be easily measurable in the system. For instance, search engines care about the number of users, how long they spend on the site, and their overall market share. Comparison statistics are not readily available to the live system. So they will need to approximate the ultimate business metric of market share with measurable ones like number of unique visitors per day and average session length. In practice, short-term, measurable live metrics may not always align with long-term business metrics, and it can be tricky to design the right metric.
- Backing up for a second, there are four classes of metrics to think about: business metrics, measurable live metrics, offline evaluation metrics, and training metrics. We just discussed the difference between business metrics and live metrics that can be measured. Offline evaluation metrics are things like the classification, regression, and ranking metrics we discussed previously. The training metric is the loss function that is optimized during the training process. (For example, a support vector machine optimizes a combination of the norm of the weight vector and misclassification penalties.)
- The optimal scenario is where all four of those metrics are either exactly the same or are linearly aligned with each other. The former is impossible. The latter is unlikely. So the next thing to shoot for is that these metrics always increase or decrease with each other. However, you may still encounter situations where a linear decrease in RMSE (a regression metric) does not translate to a linear increase in click-through rates. (Kohavi et al. described some interesting examples in their KDD 2012 paper.) Keep this in mind and save your efforts to optimize where it counts the most. You should always be tracking all of these metrics, so that you know when things go out of whack—usually a sign of distribution drift or software and instrumentation bugs.

###  How Much Change Counts as Real Change?
- Once you’ve settled on the metric, the next question is, how much of a change in this metric matters? This is required for picking the number of observations you need for the experiment. Like question #2, this is probably not solely a data science question but a business question. Pick a reasonable value up front and stick to it. Avoid the temptation to shift it later, as you start to see the results.

### One-Sided or Two-Sided Test?
- Making the wrong choice here could get you (almost) fired. One-sided (or one-tailed) tests only test whether the new model is better than the baseline. It does not tell you if it is in fact worse. You should always test both, unless you are confident it can never be worse, or there are zero consequences for it being worse. A two-sided (or two-tailed) test allows the new model to be either better or worse than the original. It still requires a separate check for which is the case.

### How Many False Positives Are You Willing to Tolerate?
- A false positive in A/B testing means that you’ve rejected the null hypothesis when the null hypothesis is true. In other words, you’ve decided that your model is better than the baseline when it isn’t better than the baseline. What’s the cost of a false positive? The answer depends on the application.
- In a drug effectiveness study, a false positive could cause the patient to use an ineffective drug. Conversely, a false negative could mean not using a drug that is effective at curing the disease. Both cases could have a very high cost to the patient’s health.
- In a machine learning A/B test, a false positive might mean switching to a model that should increase revenue when it doesn’t. A false negative means missing out on a more beneficial model and losing out on potential revenue increase.

### How Many Observations Do You Need?
- The number of observations is partially determined by the desired statistical power. This must be determined prior to running the test. A common temptation is to run the test until you observe a significant result. This is wrong.
- The power of a test is its ability to correctly identify the positives, e.g., correctly determine that a new model is doing well when it is in fact superior. It can be written as a formula that involves the significance level (question #5), the difference between the control and experimentation metrics (question #3), and the size of the samples (the number of observations included in the control and the experimentation group). You pick the right value for power, significance level, and the desired amount of change. Then you can compute how many observations you need in each group. 

### Is the Distribution of the Metric Gaussian?
- The vast majority of A/B tests use the t-test. But the t-test makes assumptions that are not always satisfied by all metrics. It’s a good idea to look at the distribution of your metric and check whether the assumptions of the t-test are valid.

### Are the Variances Equal?
- If the groups are split fairly (uniformly at random), the variances are probably equal. However, there could be subtle biases in your stream splitter (see question #1). Or perhaps one population is much smaller compared to the other. Welch’s t-test is a little-known alternative to the much more common Student’s t-test. Unlike Student’s t-test, Welch’s t-test does not assume equal variance. For this reason, it is a more robust alternative. Here’s what Wikipedia says about the advantages and limitations of Welch’s t-test:
- Welch’s t-test is more robust than Student’s t-test and maintains type I error rates close to nominal for unequal variances and for unequal sample sizes. Furthermore, the power of Welch’s t-test comes close to that of Student’s t-test, even when the population variances are equal and sample sizes are balanced.
- It is not recommended to pre-test for equal variances and then choose between Student’s t-test or Welch’s t-test. Rather, Welch’s t-test can be applied directly and without any substantial disadvantages to Student’s t-test as noted above. Welch’s t-test remains robust for skewed distributions and large sample sizes. Reliability decreases for skewed distributions and smaller samples, where one could possibly perform Welch’s t-test on ranked data.

### What Does the p-Value Mean?
- A small p-value does not imply a significant result. A smaller p-value does not imply a more significant result. The p-value is a function of the size of the samples, the difference between the two populations, and how well we can estimate the true means. I’ll leave the curious, statistically minded reader to digest the blog post (highly recommended!). The upshot is that, in addition to running the hypothesis test and computing the p-value, one should always check the confidence interval of the two population mean estimates. If the distribution is close to being Gaussian, then the usual standard error estimation applies. Otherwise, compute a bootstrap estimate, which we discussed in Chapter 3. This can differentiate between the two cases of “there is indeed a significant difference between the two populations” versus “I can’t tell whether there is a difference because the variances of the estimates are too high so I can’t trust the numbers.”

### Multiple Models, Multiple Hypotheses
- So you are a hard-working data scientist and you have not one but five new models you want to test. Or maybe 328 of them. Your website has so much traffic that you have no problem splitting off a portion of the incoming traffic to test each of the models at the same time. Parallel A1/.../Am/B testing, here we come!
- But wait, now you are in the situation of multiple hypothesis testing. Remember the false positive rate we talked about in question #5? Testing multiple hypotheses increases the overall false positive probability. If one test has a false positive rate of 0.05, then the probability that none of the 20 tests makes a false positive drops precipitously to (1 – 0.05)20 = 0.36. What’s more, this calculation assumes that the tests are independent. If the tests are not independent (i.e., maybe your 32 models all came from the same training dataset?), then the probability of a false positive may be even higher.

### How Long to Run the Test?
- When determining the length of a trial, it’s important to go beyond what’s known as the Novelty effect. When users are switched to a new experience, their initial reactions may not be their long-term reactions. In other words, if you are testing a new color for a button, the user may initially love the button and click it more often, just because it’s novel, or she may hate the new color and never touch it, but eventually she would get used to the new color and behave as she did before. It’s important to run the trial long enough to get past the period of the “shock of the new.”
- The metric may also display seasonality. For instance, the website traffic may behave one way during the day and another way at night, or perhaps people buy different types of clothes in the summer versus fall. It’s important to take this into account and discount foreseeable changes when collecting data for the trial.

### Catching Distribution Drift
- We introduced the notion of distribution drift in Chapter 1. Many machine learning models make a stationarity assumption, that the data looks and behaves one way for all eternity. But this is not true in practice. The world changes quickly. Nothing lasts forever. Translated into statistical terms, this means that the distribution of the data will drift from what the model was originally trained upon.
- To catch distribution drift, it’s a good idea to monitor the offline metric (used for evaluations during offline testing/prototyping) on live data, in addition to online testing. If the offline metric changes significantly, then it is time to update the model by retraining on new data.